<a href="https://colab.research.google.com/github/man-of-weirdness/MLnotebook/blob/main/notebook3bbee03806.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'vulnerable-c-source-code:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4785836%2F8103533%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240801%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240801T154700Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D884f3939ec409dd2880ccc2230b8d9973beb7c77345f89e605313fffba2ce3f37d5708f8c22b3e183cd34ba78f42b55e63ca9fb24ebc8a3c6346195f5df316af420d11dd87f2b1744c0686d70c2b31c318009fd7fd3e3fcb7b4e59457cc9681ac73503a9ebbe514343d5c22e599541959415566d6fd9077682c63bf85430bf69764b04cf0b3320d366cb482aee41df4012db61230c622395a31e253e198df209364215de4f4348a44e09c1bb258068040990e8a9a1b3480c04a48c38e36e649ff8108c2f140989b8ecabe7bad21fae748756dbf196634fa54119d949e45c9fa3bd5b21bc866410ad88026b2026520b94dd506987d63646e3dca5d5b02c151536,gemma2/keras/gemma2_2b_en/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F72244%2F85984%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240801%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240801T154700Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D36443b32b463cc3c3a08d10d2b3c00937f8782e06498e5890f92d8f978a0dc23931ca433f5ee578a88c82c6e300affa0e8152e94e25b14eee128048b93d4a3c82ddba11fbc57b0a2f97d8dd09031e2a13de62eecdd5d9fcbd7645f1f251d4c992643ea73dd1ce72de508e1fee9db026cd7e8bfd79ba9ff48f2289a96806cd6eb8a52af65a36ca6b7077db56c27ec833b564097cc37b99b5292e31febd0f77472827f5a1fc464d378a6d433242a87b8e62b29256613779117348103a19ee52ea21965c1d69a8bedb7ebf12683a720c05a6f7e1cd28c77bfed4302eed25c786cdb241bbdc4887fcb95d3e9e7f27de2b3441144f689df5f818f13387ff49fda1d9d'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')

[==================================================] 25013129 bytes downloaded
Downloaded and uncompressed: vulnerable-c-source-code
[================================================  ] 4058234880 bytes downloaded

In [ ]:
!pip install git+https://github.com/huggingface/accelerate.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install bitsandbytes

In [ ]:
!pip install keras_nlp

In [ ]:


import os
import numpy as np
import pandas as pd
import tensorflow as tf
import transformers
from tensorflow.keras.preprocessing.text import tokenizer_from_json
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split

#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # Disable GPU
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress TensorFlow warnings

# Function to load files from a directory
def load_files_from_directory(directory, label):
    file_contents = []
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        if os.path.isfile(filepath):
            with open(filepath, 'r', encoding='utf-8') as file:
                file_contents.append((file.read(), label))
    return file_contents

# Paths to the datasets
non_vulnerable_dir = '/kaggle/input/vulnerable-c-source-code/Dataset_raw/Non_vulnerable'
vulnerable_dir = '/kaggle/input/vulnerable-c-source-code/Dataset_raw/Vulnerable'



In [ ]:
# Load data
vulnerable_files = load_files_from_directory(vulnerable_dir, 1)
non_vulnerable_files = load_files_from_directory(non_vulnerable_dir, 0)

data = vulnerable_files + non_vulnerable_files
np.random.shuffle(data)

# Create a DataFrame and split into train, validation, and test sets
df = pd.DataFrame(data, columns=['text', 'label'])
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)



In [ ]:
# Path to your model
model_path = '/kaggle/input/gemma2/keras/gemma2_2b_en/1'

# Verify if the model path contains necessary files
if not all(os.path.isfile(os.path.join(model_path, f)) for f in ['config.json', 'model.weights.h5', 'tokenizer.json']):
    raise FileNotFoundError("One or more necessary model files are missing. Ensure 'config.json', 'model.weights.h5', and 'tokenizer.json' are present in the model directory.")



In [ ]:
# Load the tokenizer and model from the local directory
import json
import keras_nlp

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    print(gpus)
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
  except Exception as e:
      print(e)
      raise

#model = AutoModelForSequenceClassification.from_pretrained(model_path)
try:
    with open(os.path.join(model_path, 'tokenizer.json')) as f:
        tokenizer_config = json.load(f)

        print("Tokenizer config keys:", tokenizer_config.keys())

        # Assuming the custom GemmaTokenizer class is correctly imported
        from keras_nlp.src.models.gemma import GemmaTokenizer

        tokenizer = GemmaTokenizer.from_preset(model_path, load_weights=True)
        # Now you can use the tokenizer for tokenization
        print("Tokenizer loaded successfully")
except KeyError as e:
    print(f"KeyError encountered: {e}. Ensure that the tokenizer and model files are correctly formatted and located in the specified path.")
    raise

#Loading Keras model in .h5 format
model_file_path = os.path.join(model_path, 'model.weights.h5')
config_file_path = os.path.join(model_path, 'config.json')

# Load the model architecture
model = keras_nlp.models.GemmaCausalLM.from_preset(model_path, load_weights=False)
print("Model architecture loaded successfully")
model.summary()
# Load the model weights
if os.path.isfile(model_file_path):
    try:
        model.load_weights(model_file_path)
        print("Model weights loaded successfully")
    except Exception as e:
        print(f"Error loading model weights: {e}")
else:
    print(f"Model weights file not found at {model_file_path}")
model.backbone.enable_lora(rank=4)
model.summary()


In [ ]:

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,  # Adjust based on memory limits
    per_device_eval_batch_size=2,
    num_train_epochs=1,  # Set to 1 for faster testing
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate(tokenized_test)
print(results)
